In [ ]:
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


#Create a shortcut with Systems '22 in your own drive and run the following code.

In [ ]:
%cd gdrive/MyDrive/"Systems '22"/

/content/gdrive/.shortcut-targets-by-id/1ohwBGdC8wri16R_SjnGnOM_B619TDepK/Systems '22


In [ ]:
%ls

'Capstone Presentation.pptx'
'GCP output'/
 google-cloud-sdk-377.0.0-linux-x86_64.tar.gz
 labeled_video_content.gsheet
 Meeting/
 MER_Data/
'Metadata (Labeled data - Xiao) & Features Table'/
'Metadata (Unlabeled Data) & Features Table'/
 outputDetection/
'Raw videos(Xiao)'/
 saved_model/
 Scripts/
'Summary (Ask Your Doctor to Prescribe a YouTube Video).docx'
'Systems Project Course Guide 2021-2022 Spring_FINAL.pdf'
 text/
 video_content.csv
 videoIDs.txt
 videos/
'YouTube Health Analytics Capstone Project.gslides'
"YouTube Systems '22 - Team Bio Document.pdf"


In [ ]:
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

In [ ]:
!gcloud beta billing accounts list

API [cloudbilling.googleapis.com] not enabled on project [522309567947]. Would 
you like to enable and retry (this will take a few minutes)? (y/N)?  y

Enabling service [cloudbilling.googleapis.com] on project [522309567947]...
ERROR: (gcloud.beta.billing.accounts.list) PERMISSION_DENIED: Service Usage API has not been used in project 522309567947 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/serviceusage.googleapis.com/overview?project=522309567947 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.
- '@type': type.googleapis.com/google.rpc.Help
  links:
  - description: Google developers console API activation
    url: https://console.developers.google.com/apis/api/serviceusage.googleapis.com/overview?project=522309567947
- '@type': type.googleapis.com/google.rpc.ErrorInfo
  domain: googleapis.com
  metadata:
    consumer: projects/522309567947
    service: serviceusage

In [ ]:
BILLING_ACCOUNT_ID = '01F15F-5BC979-BE5194'

GCP_PROJECT_ID = 'stoked-dominion-341317'
PROJECT_NUMBER = '725342657078'

!gcloud beta billing projects link $GCP_PROJECT_ID --billing-account $BILLING_ACCOUNT_ID

API [cloudbilling.googleapis.com] not enabled on project [522309567947]. Would 
you like to enable and retry (this will take a few minutes)? (y/N)?  y

Enabling service [cloudbilling.googleapis.com] on project [522309567947]...
ERROR: (gcloud.beta.billing.projects.link) PERMISSION_DENIED: Service Usage API has not been used in project 522309567947 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/serviceusage.googleapis.com/overview?project=522309567947 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.
- '@type': type.googleapis.com/google.rpc.Help
  links:
  - description: Google developers console API activation
    url: https://console.developers.google.com/apis/api/serviceusage.googleapis.com/overview?project=522309567947
- '@type': type.googleapis.com/google.rpc.ErrorInfo
  domain: googleapis.com
  metadata:
    consumer: projects/522309567947
    service: serviceusage

In [ ]:
!gcloud services --project $GCP_PROJECT_ID enable ml.googleapis.com cloudbuild.googleapis.com

ERROR: (gcloud.services.enable) User [yaxint@andrew.cmu.edu] does not have permission to access projects instance [stoked-dominion-341317] (or it may not exist): Service Usage API has not been used in project 522309567947 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/serviceusage.googleapis.com/overview?project=522309567947 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.
- '@type': type.googleapis.com/google.rpc.Help
  links:
  - description: Google developers console API activation
    url: https://console.developers.google.com/apis/api/serviceusage.googleapis.com/overview?project=522309567947
- '@type': type.googleapis.com/google.rpc.ErrorInfo
  domain: googleapis.com
  metadata:
    consumer: projects/522309567947
    service: serviceusage.googleapis.com
  reason: SERVICE_DISABLED


In [ ]:
#!export GOOGLE_APPLICATION_CREDENTIALS='Scripts/stoked-dominion-341317-7a16d8bd74ec.json'

In [ ]:
SERVICE_ACCOUNT_NAME ='ytnotebook'

SERVICE_ACCOUNT_EMAIL = f'{SERVICE_ACCOUNT_NAME}@{GCP_PROJECT_ID}.iam.gserviceaccount.com'
!gcloud iam --project $GCP_PROJECT_ID service-accounts create $SERVICE_ACCOUNT_NAME
!gcloud projects add-iam-policy-binding $GCP_PROJECT_ID \
    --member serviceAccount:$SERVICE_ACCOUNT_EMAIL \
    --role=roles/editor

API [iam.googleapis.com] not enabled on project [522309567947]. Would you like 
to enable and retry (this will take a few minutes)? (y/N)?  

In [ ]:
DEFAULT_AI_PLATFORM_SERVICE_ACCOUNT = f'service-{PROJECT_NUMBER}@cloud-ml.google.com.iam.gserviceaccount.com'

!gcloud iam --project $GCP_PROJECT_ID service-accounts add-iam-policy-binding \
--role=roles/iam.serviceAccountAdmin \
--member=serviceAccount:$DEFAULT_AI_PLATFORM_SERVICE_ACCOUNT \
$SERVICE_ACCOUNT_EMAIL

In [ ]:
from google.cloud import storage
import os
import argparse
import pprint

os.environ.setdefault("GCLOUD_PROJECT", GCP_PROJECT_ID)

In [ ]:
# Instantiates a client
storage_client = storage.Client()

In [ ]:
#path of the files
VIDEO_FOLDER = "videos/"
AUDIO_FOLDER = "audios-wav/"
JSON_FOLDER="GCP output/output/"
#determine which folder to upload to GCP  
UP_LOAD_FILE_ROUTE = VIDEO_FOLDER
GCP_FOLDER="outputLabeled/"

#the name of the bucket
BUCKET_NAME ="video_list_capstoneyt_2022"


def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    #bucket_name = "video_list_capstoneyt_2022"
    #source_file_name = "../Toy_data/The_Greatest_Show.mp4"
    #destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )
# upload_blob('videos', './test.wav', 'test.wav')


In [ ]:
print(f"Your GCP_PROJECT_ID is:       {GCP_PROJECT_ID}")
print(f"Your SERVICE_ACCOUNT_NAME is: {SERVICE_ACCOUNT_NAME}")
print(f"Your BUCKET_NAME is:          {BUCKET_NAME}")

In [ ]:


def bucket_metadata(bucket_name):
    """Prints out a bucket's metadata."""
    # bucket_name = 'your-bucket-name'

    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)

    print("ID: {}".format(bucket.id))
    print("Name: {}".format(bucket.name))
    print("Storage Class: {}".format(bucket.storage_class))
    print("Location: {}".format(bucket.location))
    print("Location Type: {}".format(bucket.location_type))
    print("Cors: {}".format(bucket.cors))
    print(
        "Default Event Based Hold: {}".format(bucket.default_event_based_hold)
    )
    print("Default KMS Key Name: {}".format(bucket.default_kms_key_name))
    print("Metageneration: {}".format(bucket.metageneration))
    print(
        "Retention Effective Time: {}".format(
            bucket.retention_policy_effective_time
        )
    )
    print("Retention Period: {}".format(bucket.retention_period))
    print("Retention Policy Locked: {}".format(bucket.retention_policy_locked))
    print("Requester Pays: {}".format(bucket.requester_pays))
    print("Self Link: {}".format(bucket.self_link))
    print("Time Created: {}".format(bucket.time_created))
    print("Versioning Enabled: {}".format(bucket.versioning_enabled))
    print("Labels:")
    print.pprint(bucket.labels)


def upload_files(bucket_name):

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    file_names = os.listdir(UP_LOAD_FILE_ROUTE)
    for file1 in file_names:
        source_file_route = os.path.join(UP_LOAD_FILE_ROUTE, file1)
        print(GCP_FOLDER+ file1)
        blob = bucket.blob( GCP_FOLDER+file1)
        blob.upload_from_filename(source_file_route)
        print(
            "File {} uploaded to {}.".format(
                source_file_route, file1
            )
        )

In [ ]:
upload_files(BUCKET_NAME)